This is where I am experimenting with how to call and download the data from datahub's Covid-19 database

The location of the data site is https://datahub.io/core/covid-19#data
The files are in both JSON and CSV file format

US data is located at /core/covid-19/r/us_confirmed.cvs (or .json)
US data is located at /core/covid-19/r/us_deaths.cvs (or .json)

In [ ]:
import bs4 # BeautifulSoup an HTML parser
import csv # library for manipulating csv files (pandas will be better since that's what is used on datahub.io)
import os # library for saving the files within my systm
import re # regular expression library for finding data within the data
import requests # Downloads data from the web

In [ ]:
# This is the file name that I save to my machine from the website so I'm not puling requests all the time
file_name = "covid-19-data.html" 
# website of the data
base_url = "https://datahub.io/core/covid-19#data" 
#Can try to make an OOP to incorperate these two variables into it for doing the rest of the code

In [ ]:
## Code to search the datahub website to find and download the latest data.
## This can of course be addressed manually but for practice this code would 
## help me search for the link to download the data

## This can of course be addressed manually but for practice this code would 
## help me search for the link to download the data

if os.path.isfile(file_name): # may wish to delete this after the code has run or compare dates
    print(f"Using locally cached data from {file_name}") 
    with open(file_name, encoding='utf-8') as file:
        html_text = file.read()
else:
    print(f"Loading data from {base_url}")
    req = requests.get(base_url)
    req.raise_for_status()
    html_text = req.text
    with open(file_name, "w", encoding='utf-8') as file:
        file.write(html_text)
res = bs4.BeautifulSoup(html_text, "lxml")

In [ ]:
res

In [ ]:
links = list(res.select('.download > a'))# selects "a href" tags underneath .downlads with the CSS selectors
links #This does take specific knowlege on how this particular website is setup and not code to be used generally

In [ ]:
# Okay now I have the locations of the files... now what?  
# Website seems to start with https://datahub.io then /core/covid-19...
File_links= list(re.findall(r'"([^"]*)"', str(links)))
File_links

In [ ]:
#Next is do I set something that lets one choose what to download?  .csv? .json? .zip?
while True:
    File_Type= input ('What File Type do you wish to download? (.csv, .json, .zip)' )
    if '.csv' in File_Type or '.json' in File_Type or '.zip' in File_Type:
        File_links= set(re.findall(r'"([^"]*)'+File_Type, str(links)))
        print ('File type accepted')
        break
    else:
        print ("Wrong File Type!")
        continue

In [ ]:
for Type in File_links:
    if File_Type in Type:
        print ('http://datahub.io'+Type)

In [ ]:
# This method tries to select for css codes which start with a href and end with .csv 
# It's a more general approach to finding the file locations
if File_Type =='.csv':
    links_href=res.select("a[href$='.csv']")
elif Type =='.json':
    links_href=res.select("a[href$='.json']")
else:
    links_href=res.select("a[href$='.zip']")
links_href

In [ ]:
# Make a set (to remove duplicates) of the text within the " "
Files_href = set (re.findall(r'"([^"]*)"', str(links_href))) 
Files_href

In [ ]:
#That grabs some things which are not .csv
#this alteration will get the addresses of all the files
Files_csv = set (re.findall(r'"/core/covid-19/r/([^"]*)"', str(links_href))) 
Files_csv

In [ ]:
for address in Files_csv:
    File_path = 'https://datahub.io/core/covid-19/r/'+address
    print (File_path)   


In [ ]:
#This bit of code works to download the file to my computer to use.
for address in Files_csv:
    File_path = 'https://datahub.io/core/covid-19/r/'+ address
    response = requests.get(File_path)# downloads the csv file
    with open(os.path.join(address), 'wb') as f: # creates a file to write to
        f.write(response.content) #writes the data to the file

In [ ]:
#I should probably do something like the webpage downloader to make sure I only download if I need new data

In [ ]:
# Next step is to start combing through the data to start making some sense of it